In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
## Data Collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg

data = gutenberg.raw('shakespeare-hamlet.txt')

with open('hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Swastik\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [3]:
## Load the dataset
with open('hamlet.txt','r') as file:
    text = file.read().lower()

## Tokenizing the text & creating word indexes
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words

4818

In [4]:
## Create input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [5]:
## Demonstration of how each sentence is converted to tokens
line=text.split('\n')[0]
print(line)

line=tokenizer.texts_to_sequences([line])
print(line)

[the tragedie of hamlet by william shakespeare 1599]
[[1, 687, 4, 45, 41, 1886, 1887, 1888]]


In [6]:
## Sequence padding
max_sequence_length = max([len(x) for x in input_sequences])
print(f'The max lenth of an sentence is {max_sequence_length}')

input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))
print(input_sequences)

The max lenth of an sentence is 14
[[   0    0    0 ...    0    1  687]
 [   0    0    0 ...    1  687    4]
 [   0    0    0 ...  687    4   45]
 ...
 [   0    0    0 ...    4   45 1047]
 [   0    0    0 ...   45 1047    4]
 [   0    0    0 ... 1047    4  193]]


In [7]:
## Creating independent and target data
X,y = input_sequences[:,:-1],input_sequences[:,-1]
X

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]])

In [8]:
print(y)
y.shape

[ 687    4   45 ... 1047    4  193]


(25732,)

In [9]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
y.shape

(25732, 4818)

In [10]:
## Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
## Train LSTM RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GRU
from tensorflow.keras.callbacks import EarlyStopping

## Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

## Define earlystopping
early_stopping = EarlyStopping(monitor='val_loss', patience=99, restore_best_weights=True)

## Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 13, 100)           481800    
                                                                 
 lstm (LSTM)                 (None, 13, 150)           150600    
                                                                 
 dropout (Dropout)           (None, 13, 150)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [97]:
## Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    validation_data=(X_test,y_test),
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/100
644/644 [==============================] - 15s 19ms/step - loss: 6.9129 - accuracy: 0.0330 - val_loss: 6.6919 - val_accuracy: 0.0336
Epoch 2/100
644/644 [==============================] - 12s 18ms/step - loss: 6.4789 - accuracy: 0.0388 - val_loss: 6.7697 - val_accuracy: 0.0414
Epoch 3/100
644/644 [==============================] - 12s 18ms/step - loss: 6.3411 - accuracy: 0.0461 - val_loss: 6.7942 - val_accuracy: 0.0482
Epoch 4/100
644/644 [==============================] - 12s 18ms/step - loss: 6.1995 - accuracy: 0.0510 - val_loss: 6.8146 - val_accuracy: 0.0480
Epoch 5/100
644/644 [==============================] - 12s 18ms/step - loss: 6.0579 - accuracy: 0.0542 - val_loss: 6.8475 - val_accuracy: 0.0567
Epoch 6/100
644/644 [==============================] - 12s 19ms/step - loss: 5.9051 - accuracy: 0.0650 - val_loss: 6.8885 - val_accuracy: 0.0600
Epoch 7/100
644/644 [==============================] - 12s 18ms/step - loss: 5.7531 - accuracy: 0.0736 - val_loss: 6.9385 - val_ac

In [98]:
model.save('LSTM.h5')

c:\Users\Swastik\miniconda3\envs\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
## GRU RNN
## Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(GRU(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(total_words, activation='softmax'))

## Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 13, 100)           481800    
                                                                 
 gru_2 (GRU)                 (None, 13, 150)           113400    
                                                                 
 dropout_2 (Dropout)         (None, 13, 150)           0         
                                                                 
 gru_3 (GRU)                 (None, 100)               75600     
                                                                 
 dense_2 (Dense)             (None, 4818)              486618    
                                                                 
Total params: 1157418 (4.42 MB)
Trainable params: 1157418 (4.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
history = model.fit(
    X_train, y_train, 
    epochs=100,
    validation_data=(X_test,y_test),
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/100


644/644 [==============================] - 14s 17ms/step - loss: 7.0399 - accuracy: 0.0310 - val_loss: 6.8979 - val_accuracy: 0.0336
Epoch 2/100
644/644 [==============================] - 10s 16ms/step - loss: 6.5923 - accuracy: 0.0338 - val_loss: 6.8609 - val_accuracy: 0.0336
Epoch 3/100
644/644 [==============================] - 10s 15ms/step - loss: 6.4115 - accuracy: 0.0427 - val_loss: 6.8477 - val_accuracy: 0.0470
Epoch 4/100
644/644 [==============================] - 11s 17ms/step - loss: 6.2554 - accuracy: 0.0500 - val_loss: 6.8904 - val_accuracy: 0.0507
Epoch 5/100
644/644 [==============================] - 10s 15ms/step - loss: 6.0761 - accuracy: 0.0590 - val_loss: 6.8134 - val_accuracy: 0.0542
Epoch 6/100
644/644 [==============================] - 11s 17ms/step - loss: 5.8652 - accuracy: 0.0703 - val_loss: 6.9370 - val_accuracy: 0.0629
Epoch 7/100
644/644 [==============================] - 11s 17ms/step - loss: 5.6409 - accuracy: 0.0813 - val_loss: 6.9612 - val_

In [17]:
model.save('GRU.h5')

c:\Users\Swastik\miniconda3\envs\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
from tensorflow.keras.models import load_model #type: ignore
model = load_model('LSTM.h5')

In [33]:
model_gru = load_model('GRU.h5')

In [24]:
def predict_next_word(model, tokenizer, text, max_sequence_length):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_length:
        token_list = token_list[-(max_sequence_length-1)]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted = model.predict(token_list)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word,index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [48]:
## Prediction
input_text = 'to be bad is better than to'
max_sequence_length = model.input_shape[1]+1
print(f"Input Text: {input_text}")

next_word = predict_next_word(model, tokenizer, input_text, max_sequence_length)
print(f"Predicted word using LSTM: {next_word}")

next_word = predict_next_word(model_gru, tokenizer, input_text, max_sequence_length)
print(f"Predicted word using GRU: {next_word}")

Input Text: to be bad is better than to
1/1 [==============================] - 0s 20ms/step
Predicted word using LSTM: the
1/1 [==============================] - 0s 18ms/step
Predicted word using GRU: vs


In [ ]:
from tensorflow.keras.models import load_model #type: ignore
model = load_model('LSTM.h5')

In [46]:
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)